In [2]:
#Importy i Przygotowanie Danych
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_iris
from keras import layers, models

# Ustawienie seed dla powtarzalności
np.random.seed(42)
tf.random.set_seed(42)

print(f"Wersja TensorFlow: {tf.__version__}")



Wersja TensorFlow: 2.20.0


In [3]:
# --- Ładowanie danych (Iris)
data = load_iris()
df = pd.DataFrame(data.data, columns=data.feature_names)
df['target'] = data.target

X = df.drop(columns=['target']).values
y = df['target'].values

# Standaryzacja
# Skalowanie jest kluczowe, aby cechy miały średnią 0 i odchylenie standardowe 1.
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Zakres danych po skalowaniu jest potrzebny do poprawnego "klippowania"
# (przycinania) perturbacji adwersarialnych, aby zachować sensowne wartości.
min_val, max_val = X_scaled.min(), X_scaled.max()
print(f"Zakres danych po skalowaniu: [{min_val:.3f}, {max_val:.3f}]")

# --- Podział na zbiory treningowy i testowy ---
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Kształt zbioru treningowego: {X_train.shape}")

Zakres danych po skalowaniu: [-2.434, 3.091]
Kształt zbioru treningowego: (120, 4)


In [4]:
#Prosta sieć MLP (Multilayer Perceptron)
def create_model(input_dim):
    """Tworzy model MLP z dwiema warstwami ukrytymi."""
    model = models.Sequential([
        layers.Input(shape=(input_dim,)),
        layers.Dense(64, activation='relu'),
        layers.Dense(64, activation='relu'),
        # Warstwa wyjściowa: 3 klasy, aktywacja softmax
        layers.Dense(3, activation='softmax')
    ])
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy', # Używamy dla etykiet skalarnych (0, 1, 2)
                  metrics=['accuracy'])
    return model

model = create_model(X_train.shape[1])

#Krótki trening (Demo)
print("Rozpoczęcie treningu modelu bazowego (Model 'Clean'):")
# Używamy verbose=0, aby wyciszyć szczegóły epok w notebooku
model.fit(X_train, y_train, epochs=15, batch_size=16,
          validation_data=(X_test, y_test), verbose=0)

# Ocena modelu bazowego
res_clean_base = model.evaluate(X_test, y_test, verbose=0)
print(f"\n Dokładność modelu bazowego (Clean Test): {res_clean_base[1]:.4f}")

Rozpoczęcie treningu modelu bazowego (Model 'Clean'):

 Dokładność modelu bazowego (Clean Test): 0.8667


In [5]:
#Funkcja generująca adwersarialny przykład (FGSM)
def create_adversarial_example(model, input_data, input_label, epsilon, min_val, max_val):
    """
    Generuje adwersarialny przykład (perturbed input) metodą FGSM.

    input_data: 1D numpy array (skalowane cechy)
    input_label: skalarna etykieta (int)
    zwraca: 2D numpy array shape (1, n_features)
    """
    # 1. Konwersja na tensor z wymiarem batch
    x = tf.convert_to_tensor(input_data.reshape((1, -1)), dtype=tf.float32)
    y = tf.convert_to_tensor([input_label], dtype=tf.int32)

    with tf.GradientTape() as tape:
        tape.watch(x)
        preds = model(x)
        loss = tf.keras.losses.sparse_categorical_crossentropy(y, preds)

    grad = tape.gradient(loss, x)               # Obliczenie gradientu
    signed_grad = tf.sign(grad).numpy()         # Znak gradientu

    # 2. Generowanie przykładu adwersarialnego
    adv = x.numpy() + epsilon * signed_grad     # x' = x + epsilon * sign(gradient)

    # 3. Klippowanie do zakresu skalowanych danych
    adv = np.clip(adv, min_val, max_val)
    return adv



In [8]:
#Test na jednym przykładowym obrazie
idx = 0
orig = X_test[idx]
orig_label = y_test[idx]
epsilon_test = 0.8 # Siła ataku (epsilon)

pred_orig = model.predict(orig.reshape(1, -1), verbose=0)
# Używamy modelu bazowego do generowania ataku
adv_example = create_adversarial_example(model, orig, orig_label, epsilon_test, min_val, max_val)
pred_adv = model.predict(adv_example, verbose=0)

print("== Test FGSM na pojedynczym przykładzie ==")
print(f"Prawdziwa etykieta: {orig_label}")
print(f"Oryginalna predykcja: {np.argmax(pred_orig)} (konf: {np.max(pred_orig):.3f})")
print(f"Adwersarialna predykcja: {np.argmax(pred_adv)} (konf: {np.max(pred_adv):.3f})")
print(f"Czy atak się powiódł? {np.argmax(pred_orig) != np.argmax(pred_adv)}")

== Test FGSM na pojedynczym przykładzie ==
Prawdziwa etykieta: 0
Oryginalna predykcja: 0 (konf: 0.996)
Adwersarialna predykcja: 1 (konf: 0.511)
Czy atak się powiódł? True


In [9]:
#Wyszukiwanie przykładu o najniższej pewności
min_confidence = 1.0
best_idx_for_attack = -1
predictions = model.predict(X_test, verbose=0)
true_labels = y_test

# Przeszukujemy cały zbiór testowy, by znaleźć najsłabszy punkt
for i in range(len(X_test)):
    pred = predictions[i]
    confidence = np.max(pred)

    # Warunek: model musi poprawnie sklasyfikować przykład ORAZ mieć najmniejszą pewność
    if np.argmax(pred) == true_labels[i] and confidence < min_confidence:
        min_confidence = confidence
        best_idx_for_attack = i

if best_idx_for_attack != -1:
    idx = best_idx_for_attack
    print(f" Znaleziono najlepszy indeks dla ataku: {idx}")
    print(f"   Prawdziwa etykieta: {y_test[idx]}")
    print(f"   Pewność oryginalnej predykcji: {min_confidence:.4f}")
else:
    # W rzadkich przypadkach, gdy wszystkie przykłady mają idealną pewność lub kod jest zły
    idx = 0
    print(f"⚠️ Nie znaleziono idealnego słabego punktu. Używam indeksu domyślnego: {idx}")

#Przypisanie danych do testu
orig = X_test[idx]
orig_label = y_test[idx]

#Parametry ataku
epsilon_test = 0.5 # Zwiększamy epsilon, aby skompensować ewentualną trudność ataku

#Predykcje i Generowanie Ataku
pred_orig = model.predict(orig.reshape(1, -1), verbose=0)
adv_example = create_adversarial_example(model, orig, orig_label, epsilon_test, min_val, max_val)
pred_adv = model.predict(adv_example, verbose=0)

#Wyniki Ataku
print("\n== Test FGSM na Słabym Punkcie ==")
print(f"Prawdziwa etykieta: {orig_label}")
print(f"Oryginalna predykcja: {np.argmax(pred_orig)} (konf: {np.max(pred_orig):.3f})")
print(f"Adwersarialna predykcja: {np.argmax(pred_adv)} (konf: {np.max(pred_adv):.3f})")
print(f"Czy atak się powiódł? {np.argmax(pred_orig) != np.argmax(pred_adv)}")

 Znaleziono najlepszy indeks dla ataku: 9
   Prawdziwa etykieta: 1
   Pewność oryginalnej predykcji: 0.4834

== Test FGSM na Słabym Punkcie ==
Prawdziwa etykieta: 1
Oryginalna predykcja: 1 (konf: 0.483)
Adwersarialna predykcja: 2 (konf: 0.793)
Czy atak się powiódł? True
